<a href="https://colab.research.google.com/github/gsgh3016/Pytorch-study/blob/main/gamchan/chap03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 선형 회귀

## 가설 수립
---
- 가설: 머신 러닝에서 세우는 식
- 선형 회귀 가설: $H(x) = Wx + b$
  - $W$: 가중치(Weight) ~ 기울기
  - $b$: 편향(bias) ~ y절편

## 비용 함수
---
* 비용 함수 = 손실 함수 = 오차 함수 = 목적 함수
* 평균 제곱 오차(Mean Squared Error, MSE): 데이터와 가설의 $cost(W, b) = \frac{Σ|\vec{H} - \vec{d}| ^ 2}{N}$

## 경사하강법(Gradient Descent)
---
* cost function의 최소 &rarr; cost function은 음수가 될 수 있는가?
  * **Logarithmic Loss**: Some cost functions, like the log loss used in binary classification problems, can produce negative values. For binary classification, the log loss is defined as:
$ -\sum_{i=1}^N y_i \log(p(y_i)) + (1-y_i) \log(1-p(y_i)) $
where $y_i $ is the true label, $ p(y_i) $ is the predicted probability of class 1, and the sum is over all samples.<br>
<U>If the predicted probabilities are close to the true labels, the log loss will be a small negative number.</U><br>
If the predictions are wrong, the log terms can become large, resulting in a large negative cost.

* $W_{i+1} = W_i + \alpha \frac{\partial cost(W)}{\partial W}$
  * $\alpha$: learning rate
  * $\alpha$가 지나치게 높을 때 &rarr; 발산
  * $\alpha$가 지나치게 낮을 때 &rarr; 학습 속도가 느려짐

## 선형 회귀 구현

### 기본 셋팅

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# 같은 결과가 나오도록 랜덤 시드를 줍니다.
torch.manual_seed(1)

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

# 모델 초기화
## 가중치
W = torch.zeros(1, requires_grad=True)
## 편향
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

epochs = 2000
for epoch in range(epochs + 1):
  hypothesis = x_train * W + b  # y = ax + b 꼴 가설
  cost = torch.mean((hypothesis - y_train) ** 2)  # Error ^ 2의 평균 -> 오차 함수
  optimizer.zero_grad() # 초기화
  cost.backward()       # 역전파
  optimizer.step()      # SGD 업데이트
  if epoch % 100 == 0:
    print(f'Epoch {epoch:4d}/{epochs} W: {W.item():.3f}, b: {b.item():.3f} Cost: {cost.item():.6f}')

Epoch    0/2000 W: 0.187, b: 0.080 Cost: 18.666666
Epoch  100/2000 W: 1.746, b: 0.578 Cost: 0.048171
Epoch  200/2000 W: 1.800, b: 0.454 Cost: 0.029767
Epoch  300/2000 W: 1.843, b: 0.357 Cost: 0.018394
Epoch  400/2000 W: 1.876, b: 0.281 Cost: 0.011366
Epoch  500/2000 W: 1.903, b: 0.221 Cost: 0.007024
Epoch  600/2000 W: 1.924, b: 0.174 Cost: 0.004340
Epoch  700/2000 W: 1.940, b: 0.136 Cost: 0.002682
Epoch  800/2000 W: 1.953, b: 0.107 Cost: 0.001657
Epoch  900/2000 W: 1.963, b: 0.084 Cost: 0.001024
Epoch 1000/2000 W: 1.971, b: 0.066 Cost: 0.000633
Epoch 1100/2000 W: 1.977, b: 0.052 Cost: 0.000391
Epoch 1200/2000 W: 1.982, b: 0.041 Cost: 0.000242
Epoch 1300/2000 W: 1.986, b: 0.032 Cost: 0.000149
Epoch 1400/2000 W: 1.989, b: 0.025 Cost: 0.000092
Epoch 1500/2000 W: 1.991, b: 0.020 Cost: 0.000057
Epoch 1600/2000 W: 1.993, b: 0.016 Cost: 0.000035
Epoch 1700/2000 W: 1.995, b: 0.012 Cost: 0.000022
Epoch 1800/2000 W: 1.996, b: 0.010 Cost: 0.000013
Epoch 1900/2000 W: 1.997, b: 0.008 Cost: 0.000008

In [ ]:
# optimizer.zero_grad()가 필요한 이유

w = torch.tensor(2.0, requires_grad=True)

epochs = 20
for epoch in range(epochs + 1):
  z = 2 * w
  z.backward()
  print(f'w.grad: {w.grad}')

w.grad: 2.0
w.grad: 4.0
w.grad: 6.0
w.grad: 8.0
w.grad: 10.0
w.grad: 12.0
w.grad: 14.0
w.grad: 16.0
w.grad: 18.0
w.grad: 20.0
w.grad: 22.0
w.grad: 24.0
w.grad: 26.0
w.grad: 28.0
w.grad: 30.0
w.grad: 32.0
w.grad: 34.0
w.grad: 36.0
w.grad: 38.0
w.grad: 40.0
w.grad: 42.0


# 자동 미분

In [ ]:
import torch

w = torch.tensor(2.0, requires_grad=True)

y = w ** 2
z = 2 * y + 5

z.backward()

print(f'w.grad: {w.grad}')

w.grad: 8.0


# 다중 선형 회귀

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
x1_train = torch.FloatTensor([[73],
                              [93],
                              [89],
                              [96],
                              [73]])
x2_train = torch.FloatTensor([[80],
                              [88],
                              [91],
                              [98],
                              [66]])
x3_train = torch.FloatTensor([[75],
                              [93],
                              [90],
                              [100],
                              [70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

In [ ]:
X_train = torch.concat([x1_train, x2_train, x3_train], dim=1)
print(X_train.size())

torch.Size([5, 3])


In [ ]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
W = torch.zeros(3, 1, requires_grad = True)
b_2 = torch.zeros(1, requires_grad=True)

In [ ]:
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)
optimizer_2 = optim.SGD([W, b_2], lr=1e-5)

epochs = 1000
for epoch in range(epochs + 1):
  hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
  hypothesis_2 = torch.mm(X_train, W) + b_2
  cost = torch.mean((hypothesis - y_train) ** 2)
  cost_2 = torch.mean((hypothesis_2 - y_train) ** 2)

  optimizer.zero_grad()
  optimizer_2.zero_grad()

  cost.backward()
  cost_2.backward()

  optimizer.step()
  optimizer_2.step()

  if epoch % 100 == 0:
    print(f"epoch {epoch:4d}/{epochs} w1  : {w1.item():.3f} w2  : {w2.item():.3f} w3  : {w3.item():.3f} b: {b.item():.3f} Cost: {cost.item():.6f}")
    print(f"epoch {epoch:4d}/{epochs} W[0]: {W[0].item():.3f} W[1]: {W[1].item():.3f} W[2]: {W[2].item():.3f} b: {b_2.item():.3f} Cost: {cost_2.item():.6f}")

epoch    0/1000 w1  : 0.294 w2  : 0.294 w3  : 0.297 b: 0.003 Cost: 29661.800781
epoch    0/1000 W[0]: 0.294 W[1]: 0.294 W[2]: 0.297 b: 0.003 Cost: 29661.800781
epoch  100/1000 w1  : 0.674 w2  : 0.661 w3  : 0.676 b: 0.008 Cost: 1.563628
epoch  100/1000 W[0]: 0.674 W[1]: 0.661 W[2]: 0.676 b: 0.008 Cost: 1.563628
epoch  200/1000 w1  : 0.679 w2  : 0.655 w3  : 0.677 b: 0.008 Cost: 1.497595
epoch  200/1000 W[0]: 0.679 W[1]: 0.655 W[2]: 0.677 b: 0.008 Cost: 1.497595
epoch  300/1000 w1  : 0.684 w2  : 0.649 w3  : 0.677 b: 0.008 Cost: 1.435044
epoch  300/1000 W[0]: 0.684 W[1]: 0.649 W[2]: 0.677 b: 0.008 Cost: 1.435044
epoch  400/1000 w1  : 0.689 w2  : 0.643 w3  : 0.678 b: 0.008 Cost: 1.375726
epoch  400/1000 W[0]: 0.689 W[1]: 0.643 W[2]: 0.678 b: 0.008 Cost: 1.375726
epoch  500/1000 w1  : 0.694 w2  : 0.638 w3  : 0.678 b: 0.009 Cost: 1.319507
epoch  500/1000 W[0]: 0.694 W[1]: 0.638 W[2]: 0.678 b: 0.009 Cost: 1.319507
epoch  600/1000 w1  : 0.699 w2  : 0.633 w3  : 0.679 b: 0.009 Cost: 1.266222
epoc

# `nn.Module`로 구현하는 선형 회귀

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [ ]:
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1,1)

In [ ]:
print(list(model.parameters()))
# 첫번째 값이 W, 두번째 값이 b에 해당함 -> 랜덤 초기화

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 2000
for epoch in range(epochs + 1):
  prediction = model(x_train)
  cost = F.mse_loss(prediction, y_train)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print(f"Epoch {epoch:4d}/{epochs} Cost: {cost.item():.6f}")

Epoch    0/2000 Cost: 13.103541
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


In [ ]:
new_var = torch.FloatTensor([[4.]])
pred_y = model(new_var)

print(f"예측 값: {pred_y}")

예측 값: tensor([[7.9989]], grad_fn=<AddmmBackward0>)


In [ ]:
params = list(model.parameters())
print(f"# of parameters: {len(params)}")
for param in params:
  print(param)

# of parameters: 2
Parameter containing:
tensor([[1.9994]], requires_grad=True)
Parameter containing:
tensor([0.0014], requires_grad=True)


In [ ]:
torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [ ]:
# 모델을 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1. 변수의 개수임!
model = nn.Linear(3,1)

In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119]], requires_grad=True), Parameter containing:
tensor([0.2710], requires_grad=True)]


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

epochs = 2000
for epoch in range(epochs + 1):
  prediction = model(x_train)
  cost = F.mse_loss(prediction, y_train)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print(f"Epoch {epoch:4d}/{epochs} Cost: {cost.item():.6f}")

Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210560
Epoch 1100/2000 Cost: 0.209232
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206761
Epoch 1400/2000 Cost: 0.205619
Epoch 1500/2000 Cost: 0.204522
Epoch 1600/2000 Cost: 0.203484
Epoch 1700/2000 Cost: 0.202485
Epoch 1800/2000 Cost: 0.201542
Epoch 1900/2000 Cost: 0.200635
Epoch 2000/2000 Cost: 0.199769


In [ ]:
new_var = torch.FloatTensor([73, 80, 75])
pred_y = model(new_var)
print(f"예측 값: {pred_y}")

예측 값: tensor([151.2305], grad_fn=<AddBackward0>)


In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.9778, 0.4539, 0.5768]], requires_grad=True), Parameter containing:
tensor([0.2802], requires_grad=True)]


# 클래스로 파이토치 모델 구현

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [ ]:
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(1, 1)

  def forward(self, x):
    return self.linear(x)

In [ ]:
model = LinearRegressionModel()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epochs = 2000
for epoch in range(epochs + 1):
  prediction = model(x_train)
  cost = F.mse_loss(prediction, y_train)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print(f"Epoch {epoch:4d}/{epochs} Cost: {cost.item():.6f}")

Epoch    0/2000 Cost: 13.103541
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


In [ ]:
torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [ ]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [ ]:
model = MultivariateLinearRegressionModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)
epochs = 2000
for epoch in range(epochs + 1):
  prediction = model(x_train)

  cost = F.mse_loss(prediction, y_train)

  optimizer.zero_grad()

  cost.backward()

  optimizer.step()

  if epoch % 100 == 0:
    print(f"Epoch {epoch:4d}/{epochs} Cost: {cost.item():.6f}")

Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210560
Epoch 1100/2000 Cost: 0.209232
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206761
Epoch 1400/2000 Cost: 0.205619
Epoch 1500/2000 Cost: 0.204522
Epoch 1600/2000 Cost: 0.203484
Epoch 1700/2000 Cost: 0.202485
Epoch 1800/2000 Cost: 0.201542
Epoch 1900/2000 Cost: 0.200635
Epoch 2000/2000 Cost: 0.199769


# 미니 배치와 데이터 로드
---
미니 배치를 사용하는 이유: 데이터를 나눠서 학습함으로써 한정된 메모리에 대용량 데이터를 학습하기 위함

batch size &rarr; 2의 거듭제곱 사용
  * 하드웨어 메모리 구조
  * 행렬 연산 최적화
  * batch size를 변화하기 편함
  * generalization

이터레이션 $\neq$ 에포크
이터레이션: 1 에포크동안 parameter 업데이트 횟수

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
x_train  =  torch.FloatTensor([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  90],
                               [96,  98,  100],
                               [73,  66,  70]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [ ]:
dataset = TensorDataset(x_train, y_train)

In [ ]:
dataloader = DataLoader(dataset=dataset, batch_size=2, shuffle=True)

In [ ]:
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [ ]:
epochs = 10000
for epoch in range(epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    prediction = model(x_train)

    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 10 == 0:
      print(f"Epoch: {epoch:4d}/{epochs} Batch: {batch_idx + 1}/{len(dataloader)} Cost: {cost.item():.6f}")

Epoch:    0/10000 Batch: 1/3 Cost: 4.923519
Epoch:    0/10000 Batch: 2/3 Cost: 10.387980
Epoch:    0/10000 Batch: 3/3 Cost: 9.617418
Epoch:   10/10000 Batch: 1/3 Cost: 12.559605
Epoch:   10/10000 Batch: 2/3 Cost: 4.833224
Epoch:   10/10000 Batch: 3/3 Cost: 3.692606
Epoch:   20/10000 Batch: 1/3 Cost: 1.717518
Epoch:   20/10000 Batch: 2/3 Cost: 15.171709
Epoch:   20/10000 Batch: 3/3 Cost: 3.281009
Epoch:   30/10000 Batch: 1/3 Cost: 6.835446
Epoch:   30/10000 Batch: 2/3 Cost: 3.755296
Epoch:   30/10000 Batch: 3/3 Cost: 5.557797
Epoch:   40/10000 Batch: 1/3 Cost: 10.596831
Epoch:   40/10000 Batch: 2/3 Cost: 6.598134
Epoch:   40/10000 Batch: 3/3 Cost: 1.436882
Epoch:   50/10000 Batch: 1/3 Cost: 5.734137
Epoch:   50/10000 Batch: 2/3 Cost: 6.695038
Epoch:   50/10000 Batch: 3/3 Cost: 5.125523
Epoch:   60/10000 Batch: 1/3 Cost: 3.882898
Epoch:   60/10000 Batch: 2/3 Cost: 5.154518
Epoch:   60/10000 Batch: 3/3 Cost: 12.370565
Epoch:   70/10000 Batch: 1/3 Cost: 6.798835
Epoch:   70/10000 Batch: 2/

In [ ]:
new_var = torch.FloatTensor([[73, 80, 75]])
pred_y = model(new_var)
print(f"predicted  y value: {pred_y}")

predicted  y value: tensor([[151.4246]], grad_fn=<AddmmBackward0>)


# 커스텀 데이터셋
---
직접 Dataset 클래스를 디자인 &rarr; 기존 메소드 오버라이드
* `__init__(self)`: 데이터셋 생성자
* `__len__(self)`: 데이터셋의 길이 &rarr; 샘플 수
* `__getitem__(self, idx)`: 데이터셋 인덱싱

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  def __len__(self):
      return len(self.x_data)

  def __getitem__(self, idx):
      x = torch.FloatTensor(self.x_data[idx])
      y = torch.FloatTensor(self.y_data[idx])
      return x, y

In [ ]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [ ]:
epochs = 10000
for epoch in range(epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    prediction = model(x_train)

    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 1000 == 0:
      print(f"Epoch: {epoch:4d}/{epochs} Batch: {batch_idx + 1}/{len(dataloader)} Cost: {cost.item():.6f}")

Epoch:    0/10000 Batch: 1/3 Cost: 40077.906250
Epoch:    0/10000 Batch: 2/3 Cost: 13244.332031
Epoch:    0/10000 Batch: 3/3 Cost: 2370.198486
Epoch: 1000/10000 Batch: 1/3 Cost: 3.061185
Epoch: 1000/10000 Batch: 2/3 Cost: 1.908808
Epoch: 1000/10000 Batch: 3/3 Cost: 2.281282
Epoch: 2000/10000 Batch: 1/3 Cost: 0.377742
Epoch: 2000/10000 Batch: 2/3 Cost: 0.432345
Epoch: 2000/10000 Batch: 3/3 Cost: 1.577604
Epoch: 3000/10000 Batch: 1/3 Cost: 0.179620
Epoch: 3000/10000 Batch: 2/3 Cost: 0.439401
Epoch: 3000/10000 Batch: 3/3 Cost: 0.020872
Epoch: 4000/10000 Batch: 1/3 Cost: 0.094935
Epoch: 4000/10000 Batch: 2/3 Cost: 0.018611
Epoch: 4000/10000 Batch: 3/3 Cost: 0.812105
Epoch: 5000/10000 Batch: 1/3 Cost: 0.295308
Epoch: 5000/10000 Batch: 2/3 Cost: 0.154184
Epoch: 5000/10000 Batch: 3/3 Cost: 0.021614
Epoch: 6000/10000 Batch: 1/3 Cost: 0.063322
Epoch: 6000/10000 Batch: 2/3 Cost: 0.332495
Epoch: 6000/10000 Batch: 3/3 Cost: 0.261669
Epoch: 7000/10000 Batch: 1/3 Cost: 0.285514
Epoch: 7000/10000 Bat

In [ ]:
new_var = torch.FloatTensor([[73, 80, 75]])
pred_y = model(new_var)
print(f"예측 값: {pred_y}")

예측 값: tensor([[151.3084]], grad_fn=<AddmmBackward0>)
